In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical + CS1",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../data/processed/partner_ii/Type-IL.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.drop(
    [
        # Properties
        "loi",
        "water_demand",
        "setting_initial",
        "blaine",
        "sieve_32um",
        "sieve_45um",
        "CS3",
        "CS7"
    ],
    axis=1,
).copy()


<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.397 (0.063)
MAE: -1.111 (0.044)
MAPE: -0.028 (0.001)
R2: 0.715 (0.040)


******
[TEST]
******
RMSE: -2.408 (0.989)
MAE: -1.732 (0.415)
MAPE: -0.044 (0.010)
R2: -0.111 (0.854)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation  \
                                                                        
0  Chemical + Mineralogical + CS1  Linear Regression   Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  2.408091  0.988791  1.732343  0.414522  0.043661  0.010259 -0.111128   

             
        std  
0  0.853901

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -0.000 (0.000)
MAE: -0.000 (0.000)
MAPE: -0.000 (0.000)
R2: 1.000 (0.000)


******
[TEST]
******
RMSE: -6.902 (4.719)
MAE: -5.660 (3.039)
MAPE: -0.145 (0.076)
R2: -167.268 (138.520)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS1  Linear Regression   
1  Chemical + Mineralogical + CS1  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  6.901557  4.719360  5.660482  3.038815   
1              Repeated KFold  2.408091  0.988791  1.732343  0.414522   

  MAPE Test               R2 Test              
       mean       std        mean         std  
0  0.145070  0.075571 -167.268320  138.519934  
1  0.043661  0.010259   -0.111128    0.853901

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.083 (0.548)
MAE: -0.870 (0.443)
MAPE: -0.021 (0.011)
R2: 0.740 (0.139)


******
[TEST]
******
RMSE: -3.177 (1.693)
MAE: -2.619 (1.521)
MAPE: -0.066 (0.036)
R2: -5.922 (7.887)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS1  Linear Regression   
1  Chemical + Mineralogical + CS1  Linear Regression   
2  Chemical + Mineralogical + CS1  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  6.901557  4.719360  5.660482  3.038815   
1              Repeated KFold  2.408091  0.988791  1.732343  0.414522   
2           Time Series Split  3.177285  1.693379  2.619336  1.520867   

  MAPE Test               R2 Test              
       mean       std        mean         std  
0  0.145070  0.075571 -167.268320  138.519934  
1  0.043661  0.010259   -0.111128    0.853901  
2  0.065616  0.035580   -5.921551    7.886569

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 1.416 (0.000)
MAE: 1.165 (0.000)
MAPE: 0.029 (0.000)
R2: 0.720 (0.000)


******
[TEST]
******
RMSE: 1.692 (0.000)
MAE: 1.141 (0.000)
MAPE: 0.030 (0.000)
R2: -0.715 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS1  Linear Regression   
1  Chemical + Mineralogical + CS1  Linear Regression   
2  Chemical + Mineralogical + CS1  Linear Regression   
3  Chemical + Mineralogical + CS1  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  6.901557  4.719360  5.660482  3.038815   
1           Out of time Split  1.692400  0.000000  1.141378  0.000000   
2              Repeated KFold  2.408091  0.988791  1.732343  0.414522   
3           Time Series Split  3.177285  1.693379  2.619336  1.520867   

  MAPE Test               R2 Test              
       mean       std        mean         std  
0  0.145070  0.075571 -167.268320  138.519934  
1  0.030312  0.000000   -0.714999    0.000000  
2  0.043661  0.010259   -0.111128    0.853901  
3  0.065616  0.035580   -5.921551    7.886569

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([ 1.55662105e-01, -6.28628870e-04, -8.01711302e-01,  5.33977826e-01,
        3.83335716e-01, -4.43409573e-02,  6.83305865e-03, -8.06700447e-02,
        2.16400832e+00,  2.12955404e+00,  6.05866790e-01, -1.06476077e-01,
        2.13326067e-02,  2.21526229e+00])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

39.99935483870968

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['cao', 'sio2', 'al2o3', 'fe2o3', 'mgo', 'so3', 'k2o', 'na2o', 'alite',
       'belite', 'aluminate', 'ferrite', 'free_lime', 'CS1'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(14,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(14,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
CS1,2.215262
alite,2.164008
belite,2.129554
aluminate,0.605867
fe2o3,0.533978
mgo,0.383336
cao,0.155662
free_lime,0.021333
k2o,0.006833
sio2,-0.000629


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                       
0  Chemical + Mineralogical + CS1  Linear Regression   
1  Chemical + Mineralogical + CS1  Linear Regression   
2  Chemical + Mineralogical + CS1  Linear Regression   
3  Chemical + Mineralogical + CS1  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  6.901557  4.719360   
1                                {"Test Size": 0.2}  1.692400  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  2.408091  0.988791   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  3.177285  1.693379   

   MAE Test           MAPE Test               R2 Test              
       mean       std      mean       std        mean         std  
0  5.660482  3.038815  0.145070  0.075571 -167.268320  138.519934  
1  1.141378  0.000000  0.030312  0.000000   -0.714999    0.000000  
2  1.732343  0.414522  0.043661  0.010259   -0.111128    0.853901  
3  2.619336  1.520867  0.065616  0.035580   -5.921551    7.886569

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = "../../../../../reports/results/local_models/partner_ii/type-IL/full/"
filename = "linear_regression_results_full_3.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_ii/type-IL/grouped/"
filename = "linear_regression_results_grouped_3.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>